## gpu 확인 ##

```
사용환경
우분투 맥 윈도우
cuda,cudnn 설치
파이썬 3.7~3.10, 아나콘다 가상환경
```



In [ ]:
!nvidia-smi

Tue Dec 26 05:51:16 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

## Setup 라이브러리 불러오기

In [ ]:
!pip install -q peft transformers datasets evaluate
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.3 MB/s eta 0:00:00


In [ ]:
# 모델
from transformers import (
    AutoModelForSequenceClassification,
    RobertaConfig, RobertaModel, RobertaTokenizer,
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    AutoModelForCausalLM
)
# 파인튜닝
from peft import (
    LoraConfig,
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptEncoderConfig,
)
# 파이토치 데이터셋
from datasets import load_dataset
import evaluate
import torch
import os

model_name_or_path = "klue/roberta-small"
# model_name_or_path = "Hyeonseo/ko_roberta_small_model" #token classification #나쁘지않음
# model_name_or_path = "kyujinpy/KO-Platypus2-7B-ex"
# model_name_or_path = "xlm-roberta-base" #나쁘지않음

# model_name_or_path = "EleutherAI/polyglot-ko-1.3b"
# model_name_or_path = "google/mt5-base" # slow tokenizer
# model_name_or_path = "beomi/llama-2-ko-7b"
# model_name_or_path = "microsoft/phi-1_5"






## Dataset


```
로컬 환경 시 해당 경로 사용
```



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/dataset"

/content/drive/MyDrive/dataset


In [ ]:
import pandas as pd
df = pd.read_csv("./review_tags.csv", encoding="UTF-8", index_col=False)
id = pd.read_csv("./review_tag_types.csv", encoding="UTF-8", index_col=False)
df = df.loc[:,{'brand_code','review_id','message','review_tag_type_id'}]


<ipython-input-7-6e18d4b2b355>:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df = df.loc[:,{'brand_code','review_id','message','review_tag_type_id'}]


In [ ]:
id = id[id['visible']==1]
id = id[id['nlp_category_type_id']==2]
id = id.loc[:,{'id','name'}]
df, id

In [ ]:
id = id.rename(columns={'id':'review_tag_type_id' , 'name' : 'keyword'})
new_df = pd.merge(df, id, on = "review_tag_type_id")
new_df = new_df.drop({'review_tag_type_id','brand_code'},axis = 1)

new_df.isna().sum()

review_id    0
message      0
keyword      0
dtype: int64

In [ ]:
new_df = new_df[new_df['message'].notna() ]

In [ ]:
new_df = new_df.sort_values(by ='review_id').reset_index(drop = True)

## 전처리

In [ ]:
new_df = new_df.iloc[0:100000,:].drop('review_id',axis = 1)

new_df['fil'] = new_df['message']+new_df['keyword']



In [ ]:
new_df = new_df.drop_duplicates(['fil'], keep='first')
new_df= new_df.reset_index(drop= True)

new_df = new_df.drop('fil',axis = 1)

In [ ]:
leng = len(new_df)
leng80 = int(round(leng * 0.8,0))

train_df = new_df.iloc[0:leng80,:]
test_df = new_df.iloc[leng80+1:leng,:]


In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)

train_cat = ohe.fit_transform(train_df[['keyword']])
train_cat

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
ohe.categories_

[array(['가격', '기능성', '길이', '디자인', '라인(핏)', '마감처리', '배송', '사이즈', '색상', '소재',
        '스타일', '신축성', '착용감', '품질'], dtype=object)]

In [ ]:
train_df1=pd.concat([train_df.drop(columns=['keyword']),
           pd.DataFrame(train_cat, columns=[col for col in ohe.categories_[0]])], axis=1)
train_df1

,message,가격,기능성,길이,디자인,라인(핏),마감처리,배송,사이즈,색상,소재,스타일,신축성,착용감,품질
0,핏 자체도 이쁘지만 색이 너무 잘 바졌어요 ㅎ 강추합니다,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,핏 자체도 이쁘지만 색이 너무 잘 바졌어요 ㅎ 강추합니다,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,"편하고, 생각했던 재질 그대로예요!!\n근데 생각했던 것 보다 허리 둘레가 좀 커서...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"편하고, 생각했던 재질 그대로예요!!\n근데 생각했던 것 보다 허리 둘레가 좀 커서...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,"편하고, 생각했던 재질 그대로예요!!\n근데 생각했던 것 보다 허리 둘레가 좀 커서...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122972,남자친구랑 커플로 산 티셔츠인데 재질부터 색감까지 너무 예쁩니다 사이즈도 딱 맞고 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
122973,남자친구랑 커플로 산 티셔츠인데 재질부터 색감까지 너무 예쁩니다 사이즈도 딱 맞고 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
122974,남자친구랑 커플로 산 티셔츠인데 재질부터 색감까지 너무 예쁩니다 사이즈도 딱 맞고 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
122975,남자친구랑 커플로 산 티셔츠인데 재질부터 색감까지 너무 예쁩니다 사이즈는 저한테 크...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
train_df1 = train_df1.groupby('message',as_index=False).sum()
train_df1

,message,가격,기능성,길이,디자인,라인(핏),마감처리,배송,사이즈,색상,소재,스타일,신축성,착용감,품질
0,\n\n\n깔끔한 티 맘에 들어요 \n소재도 오가닉이라 더 믿음가구용\n요즘 보세티...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,\n\n\n색상 디자인 좋아요 소재도 시원하고\n가슴 부분 자수 멋지네요\n색상별로...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,\n\n\n재질이 좋아여\n짱짱해보이고\n다만 사이즈 미스때문에\n다시 시켜야할거같...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
3,\n\n\n할인해서 싼가격에 샀지만 가격대비 품질 우수하고 이쁩니다\n사이즈는 한치...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,\n\n\n화이트 블랙삿는데 예쁩니다 좀 얇은감이 살짝있고 시원하게입을수있을것같아요...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40094,힙색인데 큰편이라 덩치있는 남편이 메어도 위화감 없어요\n\n등판에 들러붙지 않아...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
40095,힙이 조금 있는 편이라서 엉덩이 부분은 타이트하고 허리 부분은 잘맞습니다. 슬림핏이...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
40096,힙이 조금 있으신 분들은 조금 타이트한 느낌을 가질수 있는 옷입니다. 전반적으로 단...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
40097,힙이 좀 큰 편이라 L시켰는데 잘 맞아요! 재질도 너무 부드러워서 입은 것 같지 않...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
test_df = test_df.reset_index(drop = True)
test_df

,message,keyword
0,남편에게 선물했는데 넘 맘에 들어해요\n원단도 너무 좋고 170에 64키로 나가는데...,라인(핏)
1,남편에게 선물했는데 넘 맘에 들어해요\n원단도 너무 좋고 170에 64키로 나가는데...,사이즈
2,남편에게 선물했는데 넘 맘에 들어해요\n원단도 너무 좋고 170에 64키로 나가는데...,소재
3,시원하고 만족해요,기능성
4,시원하고 만족해요~~~~~~~~~~~~~~~\n여름에 입기에 좋은 옷입니다.\n사이...,사이즈
...,...,...
30738,"약간 큰듯, 소재감 부드럽고 좋아요. 얇음",사이즈
30739,"약간 큰듯, 소재감 부드럽고 좋아요. 얇음",라인(핏)
30740,오버핏이여서 어깨는 넓고 사이즈는 딱 좋네요.\n기장은 95인데 어깨는 100이여서...,사이즈
30741,오버핏이여서 어깨는 넓고 사이즈는 딱 좋네요.\n기장은 95인데 어깨는 100이여서...,라인(핏)


In [ ]:

test_cat = ohe.fit_transform(test_df[['keyword']])
test_cat

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [ ]:
test_df1=pd.concat([test_df.drop(columns=['keyword']),
           pd.DataFrame(test_cat, columns=[col for col in ohe.categories_[0]])], axis=1)
test_df1

,message,가격,기능성,길이,디자인,라인(핏),마감처리,배송,사이즈,색상,소재,스타일,신축성,착용감,품질
0,남편에게 선물했는데 넘 맘에 들어해요\n원단도 너무 좋고 170에 64키로 나가는데...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,남편에게 선물했는데 넘 맘에 들어해요\n원단도 너무 좋고 170에 64키로 나가는데...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,남편에게 선물했는데 넘 맘에 들어해요\n원단도 너무 좋고 170에 64키로 나가는데...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,시원하고 만족해요,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,시원하고 만족해요~~~~~~~~~~~~~~~\n여름에 입기에 좋은 옷입니다.\n사이...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30738,"약간 큰듯, 소재감 부드럽고 좋아요. 얇음",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
30739,"약간 큰듯, 소재감 부드럽고 좋아요. 얇음",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30740,오버핏이여서 어깨는 넓고 사이즈는 딱 좋네요.\n기장은 95인데 어깨는 100이여서...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
30741,오버핏이여서 어깨는 넓고 사이즈는 딱 좋네요.\n기장은 95인데 어깨는 100이여서...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_df1 = test_df1.groupby('message',as_index=False).sum()
test_df1

,message,가격,기능성,길이,디자인,라인(핏),마감처리,배송,사이즈,색상,소재,스타일,신축성,착용감,품질
0,\n\n\n125센치 22키로 130이 맞을것 같은데 \n겨울에 키클껄 생각해서 한...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,\n\n\n탑텐몰은 정사이즈가 답이다~!넉넉하게 95시켰는데 \n왠걸~~!!아빠옷 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
2,\n\n좋아요~~~^^역시즌 아이들 겨울 패딩 준비 폴햄에서 득템~!!이 퀄리티에...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,\n\n좋아요~~~^^역시즌으로 아이들 겨울옷 장만!완전득템했어요~~~130입는 우...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,\n\n탑텐몰은 정사이즈가 답이다~!생각하고90시켰는데 \n왠걸~~!!민소매랑 속옷...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10068,흰티 안에 입으려고 산건데 한여름에 입어도 전혀 덥지않고 시원합니다\n이너용으로 최...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10069,"흰티가 필요해서 샀는데, 오버핏으로 넉넉히 입을수 있어서 좋습니다 재질도 좀 달라 ...",0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10070,"흰티에 작은 패턴이나 레터링 옷을 찾고있었는데, 아주 만족스럽습니다. 얇으면서도 신...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
10071,힙 해보이고 잘어울려서 만족 \n불편함없구 넘 만족해요,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
target_list = ['가격', '기능성', '길이', '디자인', '라인(핏)', '마감처리', '배송', '사이즈', '색상', '소재', '스타일', '신축성', '착용감', '품질']

In [ ]:
MAX_LEN = 512 #this model allow 512 token size
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 1e-05


In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['message']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
train_dataset = CustomDataset(train_df1, tokenizer, MAX_LEN)
test_dataset = CustomDataset(test_df1, tokenizer, MAX_LEN)

In [ ]:
test_dataset[0]

{'input_ids': tensor([    0, 12693,  2563,  2225,  4107,  2089,  2200,  8898,  2052,  1047,
          2069,  2728,   555,  2073,  2147,  4738,  2170,  1754,  2583,  3061,
          3628,  6396,  1891,  2225,  2113,  1414,  2371,  2203,  2182,    97,
            97, 21750,  2112, 12599,  6000,  6992,  2203,  2182,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [ ]:

train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)


In [ ]:
test_data_loader

## 모델

In [ ]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
import shutil
import numpy as np

In [ ]:

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = RobertaModel.from_pretrained(model_name_or_path, return_dict=True)
        self.dropout = torch.nn.Dropout(0.2)
        self.linear = torch.nn.Linear(768, 14)  #using 5 because we have 5 label classes

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTClass(
  (bert_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
#loss function using BCEwithlogistsloss
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:

val_targets=[]
val_outputs=[]

In [ ]:
# model.state_dict()
# new_state_dict = {key: value for key, value in model.state_dict().items() if key != 'linear.weight' and key != 'linear.bias'}
# # 새로운 딕셔너리 출력
# print(new_state_dict)

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, True, checkpoint_path, best_model_path)
      new_state_dict = {key: value for key, value in model.state_dict().items()}    # if key != 'linear.weight' and key != 'linear.bias'}
      torch.save(new_state_dict, '/content/drive/MyDrive/best_model_weights.pt')
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        torch.save(new_state_dict, '/content/drive/MyDrive/best_model_weights.pt')
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model


In [ ]:
ckpt_path = "/content/drive/MyDrive/curr_ckpt"
best_model_path = "/content/drive/MyDrive/best_model.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, test_data_loader, model, optimizer, ckpt_path, best_model_path)


############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.000509 	Average Validation Loss: 0.001161
Validation loss decreased (inf --> 0.001161).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.000251 	Average Validation Loss: 0.000758
Validation loss decreased (0.001161 --> 0.000758).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import classification_report


In [ ]:

trained_model.eval()

BERTClass(
  (bert_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
def validation(epoch):
    # model.eval()
    fin_targets=[]
    fin_outputs=[]
    softm = torch.nn.Softmax(dim=1)
    with torch.no_grad():
        for _, data in enumerate(test_data_loader, 0): # test데이터로더명

          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          targets = data['targets'].to(device, dtype = torch.float)
          # outputs = model(ids.squeeze(), mask.squeeze(), token_type_ids.squeeze())
          # outputs = model((ids.squeeze(), mask.squeeze(), token_type_ids.squeeze()))
          outputs = trained_model(ids, mask, token_type_ids)
          # fin_targets.extend(targets.cpu().detach().numpy().tolist())
          fin_targets.extend(targets.cpu().detach().numpy().tolist())

          fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets


In [ ]:
outputs, targets = validation(1)

In [ ]:
outputs = [[round(outputs[x][i]) for i in range(0,14)] for x in range(0,len(outputs))]
len(outputs), len(targets)

(10073, 10073)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

pprint.pprint(classification_report(targets,np.array(outputs,dtype=np.int)))

('              precision    recall  f1-score   support\n'
 '\n'
 '           0       0.98      0.99      0.98      3094\n'
 '           1       0.96      0.98      0.97      2731\n'
 '           2       0.97      0.96      0.96       973\n'
 '           3       0.89      0.91      0.90      2354\n'
 '           4       0.89      0.89      0.89      2495\n'
 '           5       0.91      0.63      0.75       130\n'
 '           6       0.97      0.98      0.98       697\n'
 '           7       0.96      0.98      0.97      4305\n'
 '           8       0.97      0.96      0.97      2606\n'
 '           9       0.96      0.94      0.95      4694\n'
 '          10       0.88      0.83      0.85      1504\n'
 '          11       0.94      0.90      0.92       398\n'
 '          12       0.95      0.93      0.94      3042\n'
 '          13       0.89      0.85      0.87      1720\n'
 '\n'
 '   micro avg       0.94      0.94      0.94     30743\n'
 '   macro avg       0.94      0.91      0.9

<ipython-input-46-d4992499b619>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pprint.pprint(classification_report(targets,np.array(outputs,dtype=np.int)))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 업로드

In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
model_name_or_path = 'klue/roberta-small'
config = RobertaConfig.from_pretrained(model_name_or_path)
model = RobertaModel(config=config)
print(model)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-5): 6 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout)

In [ ]:
import torch
from transformers import RobertaModel, RobertaConfig

model_name = 'klue/roberta-small'

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = RobertaModel.from_pretrained(model_name, return_dict=True)
        self.dropout = torch.nn.Dropout(0.2)
        self.linear = torch.nn.Linear(768, 14)  #using 5 because we have 5 label classes

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

    def push(self,path):
        self.bert_model.push_to_hub(path,  create_pr=1, use_auth_token = True)




model = BERTClass()
# 모델 가중치 불러오기
# model = RobertaModel.from_pretrained(model_name_or_path, return_dict=True)
model_dict = torch.load('./best_model_weights.pt')

# print(model_dict)
# model에 새로운 가중치 로드
model.load_state_dict(model_dict)
model


Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTClass(
  (bert_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push("dongseon/cllama_keyword")

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:853: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/272M [00:00<?, ?B/s]

## test

In [ ]:
# key = "가격"
# txt = "단추가 짱짱해요 "
# txt = "화면에서는 하늘색인데 진한 파랑색입니다.. 근데 세탁 후 보풀이 생겼어요"
txt = "배송이 너무너무 늦어요, 그래도 가격은 저렴하게 샀어요"
# txt = "생각보다 더 따뜻하고 프린팅도 귀엽고 색감도 이쁘고 잘산것 같아요"
# txt = "흰옷이라서 속이 좀 비치지만 몸에 딱 맞아요"
# txt = "배송이 최악이애요"

inputs = tokenizer(txt

        # f'### review: {txt} ### keyword: {key} '

    , return_tensors="pt")
# del inputs['attention_mask']

In [ ]:
inputs


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
# softm = torch.nn.Softmax(dim=1)
fin_outputs=[]
with torch.no_grad():
     # 모델이 있는 디바이스로 입력 데이터 이동
    inputs = {key: value.to(device) for key, value in inputs.items()}
    # 모델 호출
    ids = inputs['input_ids'].to(device, dtype = torch.long)
    mask = inputs['attention_mask'].to(device, dtype = torch.long)
    token_type_ids = inputs['token_type_ids'].to(device, dtype = torch.long)
    outputs = trained_model(ids, mask, token_type_ids)
    fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())


In [ ]:
fin_outputs

[[0.9691582322120667,
  0.005610233172774315,
  0.006224624812602997,
  0.011245891451835632,
  0.008015989325940609,
  0.010196688584983349,
  0.9878334999084473,
  0.014983088709414005,
  0.014843413606286049,
  0.011129903607070446,
  0.007920484058558941,
  0.003978283144533634,
  0.006165207363665104,
  0.03101613186299801]]

In [ ]:
target_list

['가격',
 '기능성',
 '길이',
 '디자인',
 '라인(핏)',
 '마감처리',
 '배송',
 '사이즈',
 '색상',
 '소재',
 '스타일',
 '신축성',
 '착용감',
 '품질']